# 第 10 章 自然语言处理入门

##10.1 分词

### 10.1.1 英文分词

In [1]:
import tensorflow.keras.preprocessing.text as kp_text

paragraph = "The 5 biggest countries by population in 2017 are China, " \
            "India, United States, Indonesia, and Brazil."
processed_text = kp_text.text_to_word_sequence(paragraph)
print(processed_text)

# 输出如下：
# ['the', '5', 'biggest', 'countries', 'by', 'population', 'in', '2017',
#  'are', 'china', 'india', 'united', 'states', 'indonesia', 'and', 'brazil']

['the', '5', 'biggest', 'countries', 'by', 'population', 'in', '2017', 'are', 'china', 'india', 'united', 'states', 'indonesia', 'and', 'brazil']


### 10.1.2 中文分词

In [2]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所")  # 搜索引擎模式
print(", ".join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/k_/86_kl60s3vg28cg5x8_b0yb80000gn/T/jieba.cache
Loading model cost 0.790 seconds.
Prefix dict has been built successfully.


我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所


## 10.2 语言模型

### 10.2.1 独热编码

In [3]:
token2idx = {'人工智能': 0, '的': 1, '研究': 2, '可以': 3, '分为': 4, '几个': 5,
             '技术': 6, '问题': 7, '是': 8, '一门': 9, '新': 10, '科学': 11}

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

samples = ["人工智能 的 研究 可以 分为 几个 技术 问题", "人工智能 是 一门 新 的 技术 科学"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(samples)

print(tokenizer.word_index)

sequence = tokenizer.texts_to_sequences(samples)
print(sequence)

print(to_categorical(sequence[0],
                     num_classes=len(tokenizer.word_index)+1))

{'人工智能': 1, '的': 2, '技术': 3, '研究': 4, '可以': 5, '分为': 6, '几个': 7, '问题': 8, '是': 9, '一门': 10, '新': 11, '科学': 12}
[[1, 2, 4, 5, 6, 7, 3, 8], [1, 9, 10, 11, 2, 3, 12]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


### 10.2.2 词嵌入

In [6]:
import gensim

model_path = 'data/word2vec/sgns.weibo.bigram-char'
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path)
vector = w2v['猫咪']
print(vector)

[ 0.102539  0.245417  0.530431  0.180411  0.386218  0.82535   0.530675
 -0.244311 -0.681341  0.431361 -0.576909 -0.231161  0.401618 -0.22292
 -0.063599  0.169365  0.282321  0.535496  0.825031 -0.049239  0.327087
  0.460255 -0.103537  0.183484  0.578723 -0.01837   0.362876  0.538694
  0.035194  0.725767 -0.414246 -0.667796 -0.122474  0.794132 -0.168633
  0.284308 -0.475918 -0.161586 -1.174823  0.725582  0.541358  0.473443
 -0.643121  0.03602  -0.3345   -0.939873 -0.752846 -0.4204    0.300105
 -0.141766 -0.224159  0.538478 -0.040462 -0.832016  0.049291  0.123678
 -0.030749  0.070804  0.451031  0.027303 -0.247646  0.021237  0.53187
 -0.007717 -0.115273  0.268562  0.472504 -0.316475  0.554178  0.109384
 -0.149233 -0.179639  0.640718 -0.260974  0.125321  0.089919 -0.64585
  0.375543  0.253541  0.142418  0.568716 -0.197845 -0.256447  0.389201
  0.037378 -0.301636  0.639302 -0.686622 -0.146073 -0.041729 -0.591598
  0.347904  0.313897 -0.508647 -0.027256 -0.121522  0.335613 -0.196134
 -0.46596

In [15]:
# 输出词向量词表前 20 个词语
print(f"word list: {w2v.index2word[:20]}")

# 输出词向量前 20 个词的向量，形状为 (20, 300)
print(f"word vectors: {w2v.vectors[:20]}")

vector = w2v['猫咪']
print(f"most similiar to 猫咪: \n {w2v.similar_by_vector(vector)}")

print(f"most similiar to 明星: \n {w2v.most_similar('明星')}")

word list: ['，', '的', '。', '@', '！', '了', '一个', '：', '我们', '、', '有', '】', '一', '是', '和', '？', '微博', '不', '“', '�']
word vectors: [[ 0.325048  0.897771 -0.418062 ... -0.47089  -0.11149   0.315512]
 [ 0.440554 -0.417241  0.1474   ... -0.235042 -0.124166  0.566537]
 [ 0.516277 -0.002949 -0.456919 ...  0.332332  0.050709  0.471254]
 ...
 [ 0.53585   0.344069  0.572108 ...  0.02912   0.009632  0.111846]
 [ 0.399981  0.189998  0.033076 ...  0.448605 -0.498731  1.144464]
 [ 0.144157  0.631021  0.475352 ...  0.715005 -0.632832 -0.617412]]
most similiar to 猫咪: 
 [('猫咪', 1.0000001192092896), ('狗狗', 0.6089414358139038), ('猫', 0.561676025390625), ('猫猫', 0.5580824613571167), ('喵', 0.4925574064254761), ('喵星人', 0.48239561915397644), ('主人', 0.45755279064178467), ('狗', 0.4399756193161011), ('宝宝', 0.4378788471221924), ('兔子', 0.4181772470474243)]
most similiar to 明星: 
 [('名人', 0.43296903371810913), ('笑星', 0.4187435507774353), ('好莱坞', 0.39082780480384827), ('女神', 0.383768767118454), ('大牌', 0.3832786083221

In [10]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(input_dim=1000,  # 标记个数，这个嵌入层总共能嵌入 999 个标记
                            output_dim=128)  # 嵌入维度

### 10.2.3 从文本到词嵌入

In [11]:
import gensim
import numpy as np
from typing import List
from tensorflow import keras

model_path = 'data/word2vec/sgns.weibo.bigram-char'
# 通常预训练词嵌入会比较大，加载很耗时耗内存资源，当内存资源有限或者需要快速实验时
# 可以通过增加一个 limit 参数，只读取特定数量词向量来节省时间和资源
# 下面代码只会加载最高频的 5000 个词的向量
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path, limit=5000)

token2index = {
    '<PAD>': 0, # 由于我们用 0 补全序列，所以补全标记的索引必须为 0
    '<UNK>': 1  # 新词标记的索引可以使任何一个，设置为 1 只是为了方便
}

# 我们遍历预训练词嵌入的词表，加入到我们的标记索引词典
for token in w2v.index2word:
    token2index[token] = len(token2index)

# 初始化一个形状为 [标记总数，预训练向量维度] 的全 0 张量
token_vector = np.zeros((len(token2index), w2v.vector_size))
# 随机初始化 <UNK> 标记的张量
token_vector[1] = np.random.rand(300)
# 从索引 2 开始使用预训练的向量
token_vector[2:] = w2v.vectors

# 通过测试可以确定新构建的标记索引和标记向量映射关系没问题
print(token_vector[token2index['成长']] == w2v['成长'])
print(token_vector[token2index['市场']] == w2v['市场'])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [12]:
# 使用处理过的预训练向量来初始化嵌入层
L = keras.layers
embedding_layer = L.Embedding(input_dim=len(token2index),  # 标记数量等于词表标记数量
                              output_dim=w2v.vector_size,  # 嵌入维度等于预训练向量维度
                              weights=[token_vector],        # 使用我们构建的权重张量
                              trainable=False)             # 不可训练
# 构建一个提取序列向量的模型
model = keras.Sequential([
    embedding_layer
])
# 我们不需要训练这个模型，所以这里的损失函数和优化器可以随意设定
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         1500600   
Total params: 1,500,600
Trainable params: 0
Non-trainable params: 1,500,600
_________________________________________________________________


In [13]:
def convert_token_2_idx(tokenized_sentence: List[str]) -> List[int]:
    """转换分词后的标记序列为标记索引序列

    如果该标记在词表出现过使用其索引，如果词表不存在，则使用新词标记的索引来替代
    Args:
        tokenized_sentence: 分词后的序列
    Returns:
        标记索引序列
    """
    token_ids = []
    for token in tokenized_sentence:
        token_ids.append(token2index.get(token, token2index['<UNK>']))
    return token_ids

tokenized_sentence = "今天 天气 真 不错 ha".split(' ')
print(convert_token_2_idx(tokenized_sentence))

[89, 438, 462, 242, 1]


In [14]:
sentence_index = convert_token_2_idx(tokenized_sentence)
# 将序列索引包含一个样本的批量
input_x = np.array([sentence_index])
# 使用模型预测
sentence_vector = model.predict(input_x)
print(sentence_vector.shape)

(1, 5, 300)
